## Problem 5

In [ ]:
## For this question, fill in the TODO's 

In [8]:
!pip install git+https://github.com/adapter-hub/adapter-transformers.git
!git clone https://github.com/huggingface/transformers
!python transformers/utils/download_glue_data.py --tasks RTE
!pip install adapter-transformers==1.0.1
import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, Optional

import numpy as np

import torch
from transformers import AutoTokenizer, EvalPrediction, GlueDataset, GlueDataTrainingArguments, AutoModelWithHeads, AdapterType, AutoConfig, AutoModelForSequenceClassification
from transformers import GlueDataTrainingArguments as DataTrainingArguments
from transformers import (
    Trainer,
    TrainingArguments,
    glue_compute_metrics,
    glue_tasks_num_labels,
    set_seed,
)

model_name = "bert-base-uncased"


# Refer to the notebook for training an adapter to write these. Set the number of epochs to 3, and learning rate to 5e-5. Rest of the hyperparameters can stay the same. 

data_args = GlueDataTrainingArguments(task_name="rte", data_dir="./glue_data/RTE")

training_args = TrainingArguments(
    logging_steps=50, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=64, 
    save_steps=1000,
    evaluate_during_training=True,
    output_dir="./models/rte",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    learning_rate=5e-5,
    num_train_epochs=6,
)

# TODO: Change this seed when re-running your code to report the mean and std dev
set_seed(100)
num_labels = glue_tasks_num_labels[data_args.task_name]


  Cloning https://github.com/adapter-hub/adapter-transformers.git to /tmp/pip-req-build-dljfogzx
  Running command git clone -q https://github.com/adapter-hub/adapter-transformers.git /tmp/pip-req-build-dljfogzx
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached https://files.pythonhosted.org/packages/4c/34/b39eb9994bc3c999270b69c9eea40ecc6f0e97991dba28282b9fd32d44ee/tokenizers-0.9.3-cp36-cp36m-manylinux1_x86_64.whl
  Created wheel for adapter-transformers: filename=adapter_transformers-1.1.0-cp36-none-any.whl size=1326572 sha256=da3bf7a4d1b3abb716b0068ae85ee85e2740cc4f55add41d15502584c1ef8d8e
  Stored in directory: /tmp/pip-ephem-wheel-cache-6hd3tcxg/wheels/22/eb/df/1c86c6a1b0323a74470d6a53db05b3b49ec79bce18d253ec38
Successfully built adapter-transformers
  Found existing installation: tokenizers 0.7.0
    Uninstalling tokenizers-0.7.0:
      Successfully uninstalled tokenizers-0.7.0
  Found exis

fatal: destination path 'transformers' already exists and is not an empty directory.
	Completed!
  Using cached https://files.pythonhosted.org/packages/97/ec/7a766374742566d209d63a18dc0533f2cb79def3b116b903ec7bad7ca4bf/adapter_transformers-1.0.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/14/e5/a26eb4716523808bb0a799fcfdceb6ebf77a18169d9591b2f46a9adb87d9/tokenizers-0.7.0-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: tokenizers 0.9.3
    Uninstalling tokenizers-0.9.3:
      Successfully uninstalled tokenizers-0.9.3
  Found existing installation: adapter-transformers 1.1.0
    Uninstalling adapter-transformers-1.1.0:
      Successfully uninstalled adapter-transformers-1.1.0


In [24]:
config = AutoConfig.from_pretrained(
        model_name,
        num_labels=num_labels,
        finetuning_task=data_args.task_name,
        cache_dir=".",
    )

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=".",
)

model = AutoModelWithHeads.from_pretrained(model_name, config=config)


Now we have everything set up to load our AdapterFusion setup. 

First, you need to go to adapterhub.ml and explore the different adapters available. Choose any three adapters and load the three adapters. As we don't need their prediction heads, we pass with_head=False to the loading method. Next, we add a new fusion layer that combines all the adapters we've just loaded. Finally, we add a new classification head for our target task on top.

In [25]:
# First, load the pre-trained adapters we want to fuse from Hub
from transformers.adapter_config import PfeifferConfig

model.load_adapter("nli/rte@ukp", "text_task", config=PfeifferConfig(), load_as='rte', with_head=False)
# TODO: load some more adapters. Choose the ones that you think will help RTE task after reading about the different tasks available and how big the datasets are.
#model.load_adapter("nli/sick@ukp", "text_task", config=PfeifferConfig(), load_as='sick', with_head=False)
#model.load_adapter("qa/boolq@ukp", "text_task", config=PfeifferConfig(), load_as='boolq', with_head=False)

#model.load_adapter("comsense/cosmosqa@ukp", "text_task", config=PfeifferConfig(), load_as='cosmosqa', with_head=False)
model.load_adapter("comsense/siqa@ukp", "text_task", config=PfeifferConfig(), load_as='siqa', with_head=False)
model.load_adapter("nli/cb@ukp", "text_task", config=PfeifferConfig(), load_as='cb', with_head=False)
model.load_adapter("qa/boolq@ukp", "text_task", config=PfeifferConfig(), load_as='boolq', with_head=False)

#model.load_adapter("comsense/winogrande@ukp", "text_task", config=PfeifferConfig(), load_as='winogrande', with_head=False)
#model.load_adapter("sts/mrpc@ukp", "text_task", config=PfeifferConfig(), load_as='mrpc', with_head=False)

#model.load_adapter("nli/multinli@ukp", "text_task", config=PfeifferConfig(), load_as='multiNli', with_head=False)
#model.load_adapter("nli/qnli@ukp", "text_task", config=PfeifferConfig(), load_as='qnli', with_head=False)
#model.load_adapter("nli/cb@ukp", "text_task", config=PfeifferConfig(), load_as='cb', with_head=False)

#model.load_adapter("nli/scitail@ukp", "text_task", config=PfeifferConfig(), load_as='scitail', with_head=False)
#model.load_adapter("nli/sick@ukp", "text_task", config=PfeifferConfig(), load_as='sick', with_head=False)
#model.load_adapter("nli/cb@ukp", "text_task", config=PfeifferConfig(), load_as='cb', with_head=False)

#
# Add a fusion layer and tell the model to train fusion (freezes the rest of the weights) (here can either add the actual atsk adapter or not)
model.add_fusion([
        'rte',
        'siqa',
         'cb',
        'boolq'
        #,'mrpc'
        
        #"scitail"
        # TODO: Add your other task names here for the adapters you chose
    ])

# Add a classification head for our target task
# TODO: Check the earlier notebook from Problem 5 to see how to add a classification head for your task.
model.add_classification_head("rte", num_labels=num_labels, overwrite_ok= True)
#model.add_classification_head("multiNli", num_labels=num_labels, overwrite_ok= True)
model.add_classification_head("siqa", num_labels=num_labels, overwrite_ok= True)
#model.add_classification_head("qnli", num_labels=num_labels, overwrite_ok= True)
#model.add_classification_head("scitail", num_labels=num_labels, overwrite_ok= True)
model.add_classification_head("cb", num_labels=num_labels, overwrite_ok= True)
#model.add_classification_head("qnli", num_labels=num_labels, overwrite_ok= True)
#model.add_classification_head("winogrande", num_labels=num_labels, overwrite_ok= True)
#model.add_classification_head("mrpc", num_labels=num_labels, overwrite_ok= True)
model.add_classification_head("blooq", num_labels=num_labels, overwrite_ok= True)

#model.set_active_adapters([["rti", "multiNli", "qnli", "cb"]])
#model.set_active_adapters([["rti", "multiNli", "qnli", "sick"]])
model.set_active_adapters([[ "siqa", "cb", "boolq"]])


The last preparation step is to define and activate our adapter setup. Similar to train_adapter(), train_fusion() does two things: It freezes all weights of the model (including adapters!) except for the fusion layer and classification head. It also activates the given adapter setup to be used in very forward pass.

The syntax for the adapter setup (which is also applied to other methods such as set_active_adapters()) works as follows:

a single string is interpreted as a single adapter, 
a list of strings is interpreted as a stack of adapters,
a nested list of strings is interpreted as a fusion of adapters. Here want to do Fusion so we use a nested list as follows. 


In [26]:
adapter_setup = [
                 [
        "rte",
        "siqa",
        "cb",
        "boolq"
        #,"mrpc"
        #"sick"
        # TODO: Add your other adapter names here. 
    ]
]
model.train_fusion(adapter_setup)

In [27]:

# Check out your training args
print(training_args)


TrainingArguments(output_dir='./models/rte', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=True, per_device_train_batch_size=32, per_device_eval_batch_size=64, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=6, max_steps=-1, warmup_steps=0, logging_dir=None, logging_first_step=False, logging_steps=50, save_steps=1000, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O1', local_rank=-1, tpu_num_cores=None, tpu_metrics_debug=False)


In [28]:
import time as t
training_args = TrainingArguments(
    logging_steps=50, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=64, 
    save_steps=1000,
    evaluate_during_training=True,
    output_dir="./models/rte",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    learning_rate=5e-5,
   # learning_rate = 1e-4,
    num_train_epochs=6,
)
seed = int(np.abs((t.time()-np.round(t.time(),4))*10000000))
#seed = 97
#seed = 100
print (seed)
set_seed(seed)

269


In [29]:
train_dataset = GlueDataset(data_args, tokenizer=tokenizer)
eval_dataset = GlueDataset(data_args, tokenizer=tokenizer, mode="dev")

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return glue_compute_metrics(data_args.task_name, preds, p.label_ids)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        do_save_full_model=False,
        do_save_adapter_fusion=True,
    )

trainer.train()
trainer.evaluate()

{"loss": 0.36211424008011817, "learning_rate": 4.465811965811966e-05, "epoch": 0.6410256410256411, "step": 50}



{"eval_loss": 0.8555555045604706, "eval_acc": 0.7328519855595668, "epoch": 0.6410256410256411, "step": 50}



{"loss": 0.19210255272686483, "learning_rate": 3.931623931623932e-05, "epoch": 1.282051282051282, "step": 100}



{"eval_loss": 0.8998389482498169, "eval_acc": 0.7292418772563177, "epoch": 1.282051282051282, "step": 100}
{"loss": 0.16334185261279344, "learning_rate": 3.397435897435898e-05, "epoch": 1.9230769230769231, "step": 150}



{"eval_loss": 0.9924060702323914, "eval_acc": 0.7184115523465704, "epoch": 1.9230769230769231, "step": 150}



...

{"loss": 0.13334948439151048, "learning_rate": 7.264957264957266e-06, "epoch": 5.128205128205128, "step": 400}



{"eval_loss": 1.021021556854248, "eval_acc": 0.7364620938628159, "epoch": 5.128205128205128, "step": 400}
{"loss": 0.1335635694861412, "learning_rate": 1.9230769230769234e-06, "epoch": 5.769230769230769, "step": 450}



{"eval_loss": 1.0190040707588195, "eval_acc": 0.740072202166065, "epoch": 5.769230769230769, "step": 450}





{"eval_loss": 1.0180757522583008, "eval_acc": 0.740072202166065, "epoch": 6.0, "step": 468}


{'epoch': 6.0, 'eval_acc': 0.740072202166065, 'eval_loss': 1.0180757522583008}

In [17]:
## without RTE
mean = np.mean([0.7328,0.7364,0.7364])
std = np.std([0.7328,0.7364,0.7364])
print (f'mean {mean}')
print (f'std {std}')

mean 0.7352
std 0.0016970562748477365


In [30]:
## with RTE
mean = np.mean([0.740072,0.7436,0.7436])
std = np.std([0.740072,0.7436,0.7436])
print (f'mean {mean}')
print (f'std {std}')

mean 0.7424240000000001
std 0.0016631151493508006
